In [ ]:
import os
import json
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import mysql.connector

In [ ]:
from pony.orm import Database, Required, Optional, Json, PrimaryKey, db_session

# Define the Pony ORM database
db = Database()

In [ ]:
# Function to extract data from TXT file
def extract_txt(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()
    return [{'text_data': line.strip()} for line in data]

In [ ]:
# Function to extract data from JSON file
def extract_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Check if data is a list, and handle accordingly
    if isinstance(data, list):
        # If data is a list, return a list of dictionaries
        return [{'json_' + key: value for key, value in item.items()} for item in data]
    else:
        # If data is a single dictionary, return a list containing that dictionary
        return [{'json_' + key: value} for key, value in data.items()]

In [ ]:
# Function to extract data from XML file
def extract_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    return extract_data_from_xml(root)

In [ ]:
# Function to extract data from XML root
def extract_data_from_xml(root):
    data = []
    for elem in root.findall('.//user'):
        user_data = {}
        user_data['xml_firstName'] = elem.attrib.get('firstName', '')
        user_data['xml_lastName'] = elem.attrib.get('lastName', '')
        user_data['xml_age'] = elem.attrib.get('age', '')
        user_data['xml_sex'] = elem.attrib.get('sex', '')
        data.append(user_data)
    return data

In [ ]:
# Function to extract data from CSV file
def extract_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        data = [{f"csv_{key}": value for key, value in row.items()} for row in reader]
    return data

In [ ]:
# Function to transform data into a homogenous record
def transform_data(data):
    transformed_data = [item for item in data if all(item.values())]
    return transformed_data

In [ ]:
# Function to combine data from different sources into one dataset
def combine_data(txt_data, json_data, xml_data, csv_data):
    max_len = max(len(txt_data), len(json_data), len(xml_data), len(csv_data))
    combined_data = []
    for i in range(max_len):
        record = {}
        record.update(txt_data[i % len(txt_data)])
        record.update(json_data[i % len(json_data)])
        record.update(xml_data[i % len(xml_data)])
        record.update(csv_data[i % len(csv_data)])
        combined_data.append(record)
    return combined_data

In [ ]:
# Function to save combined data to CSV
def save_combined_data_to_csv(combined_data, output_file):
    df = pd.DataFrame(combined_data)
    df.to_csv(output_file, index=False)

In [ ]:
# MySQL database connection
@db.on_connect(provider='mysql')
def mysql_on_connect(db, connection):
    connection.autocommit = True

In [ ]:
# Function to load data into MySQL database
def load_data_into_mysql(csv_file, user, password, host, database):
    connection = mysql.connector.connect(user=user, password=password, host=host, database=database)
    cursor = connection.cursor()

    try:
        # Create table if not exists
        create_table_query = """
        CREATE TABLE IF NOT EXISTS combined_data (
            text_data TEXT,
            json_firstName VARCHAR(255),
            json_lastName VARCHAR(255),
            json_age INT,
            json_iban VARCHAR(255),
            json_credit_card_number VARCHAR(255),
            json_credit_card_security_code VARCHAR(255),
            json_credit_card_start_date VARCHAR(10),
            json_credit_card_end_date VARCHAR(10),
            json_address_main VARCHAR(255),
            json_address_city VARCHAR(255),
            json_address_postcode VARCHAR(10),
            xml_firstName VARCHAR(255),
            xml_lastName VARCHAR(255),
            xml_age INT,
            xml_sex VARCHAR(10),
            csv_First_Name VARCHAR(255),
            csv_Second_Name VARCHAR(255),
            csv_Age_Years INT,
            csv_Sex VARCHAR(10),
            csv_Vehicle_Make VARCHAR(255),
            csv_Vehicle_Model VARCHAR(255),
            csv_Vehicle_Year INT,
            csv_Vehicle_Type VARCHAR(50)
        )
        """
        cursor.execute(create_table_query)

        # Load data from CSV into MySQL
        with open(csv_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                insert_query = """
                INSERT INTO combined_data 
                (text_data, json_firstName, json_lastName, json_age, json_iban, json_credit_card_number, 
                json_credit_card_security_code, json_credit_card_start_date, json_credit_card_end_date, 
                json_address_main, json_address_city, json_address_postcode, xml_firstName, xml_lastName, 
                xml_age, xml_sex, csv_First_Name, csv_Second_Name, csv_Age_Years, csv_Sex, 
                csv_Vehicle_Make, csv_Vehicle_Model, csv_Vehicle_Year, csv_Vehicle_Type) 
                VALUES 
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (
                    row['text_data'], row['json_firstName'], row['json_lastName'], row['json_age'], 
                    row['json_iban'], row['json_credit_card_number'], row['json_credit_card_security_code'], 
                    row['json_credit_card_start_date'], row['json_credit_card_end_date'], 
                    row['json_address_main'], row['json_address_city'], row['json_address_postcode'], 
                    row['xml_firstName'], row['xml_lastName'], row['xml_age'], row['xml_sex'], 
                    row['csv_First Name'], row['csv_Second Name'], row['csv_Age (Years)'], row['csv_Sex'], 
                    row['csv_Vehicle Make'], row['csv_Vehicle Model'], row['csv_Vehicle Year'], 
                    row['csv_Vehicle Type']
                ))

        # Commit changes and close connection
        connection.commit()
        print("Data loaded successfully into MySQL database.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        connection.close()

In [ ]:
# Main function
def main():
    # Extract data from different sources
    txt_data = extract_txt('user_data_23_4.txt')
    json_data = extract_json('user_data_23_4.json')
    xml_data = extract_xml('user_data_23_4.xml')
    csv_data = extract_csv('user_data_23_4.csv')

    # Combine data
    combined_data = combine_data(txt_data, json_data, xml_data, csv_data)

    # Save combined data to CSV
    save_combined_data_to_csv(combined_data, 'combined_data.csv')


    # Load data into MySQL database
    load_data_into_mysql(
         'combined_data.csv', # combined file
         'student_bi59ij', # username
         'iE93F2@8EhM@1zhD&u9M@K', # password
         'europa.ashley.work', # host
         'student_bi59ij'  # database
         )

# Call the main function
main()